In [9]:
import os
from string import punctuation
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

In [44]:
fill_type = ['SALINE', 'SILICONE', 'GEL', 'COHESIVE']
surface_type = ['SMOOTH', 'TEXTURED', 'BIOCELL', 'MICROCELL', 'POLYURETHANE']
implantation_indication = ['AUGMENTATION', 'RECONSTRUCTION', 'COSMETIC', 'REVISION', 'AUGMENTATION REVISION',
                          'RECONSTRUCTION REVISION']
ALCL = ['ALCL', 'ANAPLASTIC LARGE CELL LYMPHOMA', 'LYMPHOMA', 'T-CELL LYMPHOMA', 'B-CELL LYMPHOMA',
        'CANCER', 'TUMOR', 'SUSPECT', 'CONFIRMED', 'SUSPECT ALCL', 'CONFIRMED ALCL']
side = ['LEFT', 'RIGHT', 'BILATERAL', 'BOTH SIDES', 'LEFT+RIGHT']
biomarker = ['CD 30', 'CD30', 'ALK', 'NEGATIVE', 'POSITIVE', 'CD30-', 'CD30+', 'ALK-', 'ALK+', 'CD30 NEGATIVE',
            'CD30 POSITIVE', 'ALK NEGATIVE', 'ALK POSITIVE']
symptom = ['breast pain', 'breast swell', 'breast cyst', 'breast calcif', 'capsular contracture',
           'lymph node enlarge', 'firmness of breast', 'hematoma', 'mass', 'lump', 'rupture', 'deflat',
           'infect', 'abscess', 'leukopenia', 'nodule', 'skin discolor', 'skin lesion', 'seroma',
           'effusion', 'fluid', 'edema', 'leak', 'red', 'tender', 'erythema', 'asymmetr', 'breast enlarge']

df_temp = pd.read_excel('2_foidev_column_full_list_standardized.xlsx', sheetname='foidev_MANUFACTURER_full_list')

In [14]:
punctuations = '''!()[]{};:'"\,<>./?@#$%^&*_~'''
stopwords = nltk.corpus.stopwords.words('english')

def processing_text(s):
    s = str(s)
    s = s.lower()
    no_punct = ""
    for char in s:
        if char not in punctuations:
            no_punct = no_punct + char
    if ('alcl-' in no_punct) | ('gel-' in no_punct):
        no_punct = no_punct.replace('alcl-', 'alcl ')
        no_punct = no_punct.replace('alcl-', 'gel ')
    l = no_punct.split(' ')
    output = []
    for w in l:
        if w in stopwords:
            continue
        output.append(w)
    return ' '.join(output).upper()

def convert_manufacturer(m):
    std_manufacturer = df_temp.loc[df_temp['Value']==m, 'Standardized term']
    if len(std_manufacturer) == 0:
        return m
    else:
        std_manufacturer = str(std_manufacturer.values[0])
        std_manufacturer = std_manufacturer.upper()
        return std_manufacturer

In [15]:
df_BI_DEV = pd.read_csv('WF/DEV_BI_FULL_TABLE.txt', sep='|', header=0, encoding='ISO-8859-1', error_bad_lines=False)
df_BI_DEV = df_BI_DEV.drop_duplicates()
df_BI_TEXT = pd.read_csv('WF/TEXT_FULL_BI_LIST.txt', sep='|', header=0, encoding='ISO-8859-1', error_bad_lines=False)
df_BI_TEXT = df_BI_TEXT.drop_duplicates('MDR_TEXT_KEY')
print('Device #:', df_BI_DEV.shape[0])
print('Text #:', df_BI_TEXT.shape[0])
df_BI = df_BI_TEXT.merge(df_BI_DEV, on='MDR_REPORT_KEY', how='inner')
df_BI['MANUFACTURER_D_NAME'] = df_BI['MANUFACTURER_D_NAME'].astype(str)
df_BI['MANUFACTURER_D_NAME'] = df_BI['MANUFACTURER_D_NAME'].map(convert_manufacturer)
df_BI['BRAND_NAME'] = df_BI['BRAND_NAME'].astype(str)
df_BI['GENERIC_NAME'] = df_BI['GENERIC_NAME'].astype(str)
df_BI['FOI_TEXT'] = df_BI['FOI_TEXT'].astype(str)
df_BI = df_BI.drop_duplicates()
df_BI['FOI_TEXT'] = df_BI['FOI_TEXT'].map(processing_text)
print('Full table #:', df_BI.shape[0])

Device #: 28182
Text #: 27137
Full table #: 28301


In [31]:
def text_search(text, k):
    l = text.split(' ')
    if k == 'B-CELL LYMPHOMA':
        if (k in text) | ('B CELL LYMPHOMA' in text):
            return True
        else:
            return False
    elif ' ' in k:
        if k in text:
            return True
        else:
            return False
    elif k == 'LEFT+RIGHT':
        if ('LEFT' in text) & ('RIGHT' in text):
            return True
        else:
            return False
    else:
        for i in l:
            if k in i:
                return True
        return False

def filter_merge(f1, f2, f3):
    return (f1 | f2 | f3)

In [17]:
df_vector = pd.DataFrame()
df_vector['MDR_REPORT_KEY'] = df_BI['MDR_REPORT_KEY']
df_vector['MANUFACTURER_D_NAME'] = df_BI['MANUFACTURER_D_NAME']
df_vector['BRAND_NAME'] = df_BI['BRAND_NAME']
df_vector['GENERIC_NAME'] = df_BI['GENERIC_NAME']
df_vector['FOI_TEXT'] = df_BI['FOI_TEXT']

In [32]:
df_temp = pd.read_excel('2_foidev_column_full_list_standardized.xlsx', sheetname='foidev_MANUFACTURER_full_list')

print('Manufacturer word frequecy:\n')
manu_list = list(set(list(df_BI['MANUFACTURER_D_NAME'])))
manu_list.remove('nan')
manu_list.remove('NAN')
manu_list.remove('UNK')
for m in manu_list:
    df_BI['filter'] = np.vectorize(text_search)(df_BI['MANUFACTURER_D_NAME'], m)
    temp = df_BI.loc[df_BI['filter']==True, :]
    df_vector[m] = np.nan
    df_vector.loc[df_BI['filter']==True, m] = 1
    try:
        df_vector[m] = df_vector[m].fillna(0)
        df_vector[m] = df_vector[m].astype(int)
    except:
        continue
    del df_BI['filter']
    temp.to_csv('WF_FULL_TABLE/manufacturer/'+m+'.txt', header=True, sep='|', index=False)
    print(m, ':', temp.shape[0])
    del temp

Manufacturer word frequecy:

PMT CORP. : 20
ALLERGAN : 2600
COX-UPHOFF INTERNATIONAL : 128
MEDICAL ENGINEERING CORPRATION : 275
POLY IMPLANT PROTHESE : 1442
BIOPLASTY INC. : 24
INAMED : 207
MENTOR : 6968
NAGOR LTD : 2
MCGHAN : 2205
NATURAL Y SURGICAL SPECIALTIES INC. : 16
SILIMED : 13
ALLEGIANCE HEALTHCARE CORP. : 12
BAXTER HEALTHCARE CORPRATION : 486
SIENTRA : 44
DOW CORNING CORPRATION : 3728
EUROMED, INC. : 1
IDEAL IMPLANT INCORPORATED : 57
MEDICAL ENGINEERING CORPORATION : 1581
HUTCHISON INTERNATIONAL, INC. : 3
BIOSIL LTD : 1760


In [33]:
print('Surface type word frequecy:\n')
for s in surface_type:
    df_BI['filter_1'] = np.vectorize(text_search)(df_BI['BRAND_NAME'], s)
    df_BI['filter_2'] = np.vectorize(text_search)(df_BI['GENERIC_NAME'], s)
    df_BI['filter_3'] = np.vectorize(text_search)(df_BI['FOI_TEXT'], s)
    df_BI['filter'] = np.vectorize(filter_merge)(df_BI['filter_1'], df_BI['filter_2'], df_BI['filter_3'])
    temp = df_BI.loc[df_BI['filter']==True, :]
    df_vector[s] = np.nan
    df_vector.loc[df_BI['filter']==True, s] = 1
    try:
        df_vector[s] = df_vector[s].fillna(0)
        df_vector[s] = df_vector[s].astype(int)
    except:
        continue
    del df_BI['filter'], df_BI['filter_1'], df_BI['filter_2'], df_BI['filter_3']
    temp.to_csv('WF_FULL_TABLE/surface/'+s+'.txt', header=True, sep='|', index=False)
    print(s, ':', temp.shape[0])
    del temp

Surface type word frequecy:

SMOOTH : 1056
TEXTURED : 821
BIOCELL : 80
MICROCELL : 1
POLYURETHANE : 391


In [34]:
print('Fill type word frequecy:\n')
for f in fill_type:
    df_BI['filter_1'] = np.vectorize(text_search)(df_BI['BRAND_NAME'], f)
    df_BI['filter_2'] = np.vectorize(text_search)(df_BI['GENERIC_NAME'], f)
    df_BI['filter_3'] = np.vectorize(text_search)(df_BI['FOI_TEXT'], f)
    df_BI['filter'] = np.vectorize(filter_merge)(df_BI['filter_1'], df_BI['filter_2'], df_BI['filter_3'])
    temp = df_BI.loc[df_BI['filter']==True, :]
    df_vector[f] = np.nan
    df_vector.loc[df_BI['filter']==True, f] = 1
    try:
        df_vector[f] = df_vector[f].fillna(0)
        df_vector[f] = df_vector[f].astype(int)
    except:
        continue
    del df_BI['filter'], df_BI['filter_1'], df_BI['filter_2'], df_BI['filter_3']
    temp.to_csv('WF_FULL_TABLE/fill/'+f+'.txt', header=True, sep='|', index=False)
    print(f, ':', temp.shape[0])
    del temp

Fill type word frequecy:

SALINE : 8322
SILICONE : 11394
GEL : 11466
COHESIVE : 364


In [35]:
print('Implantation indication word frequecy:\n')
for i in implantation_indication:
    df_BI['filter'] = np.vectorize(text_search)(df_BI['FOI_TEXT'], i)
    temp = df_BI.loc[df_BI['filter']==True, :]
    df_vector[i] = np.nan
    df_vector.loc[df_BI['filter']==True, i] = 1
    try:
        df_vector[i] = df_vector[i].fillna(0)
        df_vector[i] = df_vector[i].astype(int)
    except:
        continue
    del df_BI['filter']
    temp.to_csv('WF_FULL_TABLE/implantation_indication/'+i+'.txt', header=True, sep='|', index=False)
    print(i, ':', temp.shape[0])
    del temp

Implantation indication word frequecy:

AUGMENTATION : 1985
RECONSTRUCTION : 1108
COSMETIC : 267
REVISION : 344
AUGMENTATION REVISION : 6
RECONSTRUCTION REVISION : 6


In [36]:
print('ALCL word frequecy:\n')
for m in ALCL:
    df_BI['filter'] = np.vectorize(text_search)(df_BI['FOI_TEXT'], m)
    temp = df_BI.loc[df_BI['filter']==True, :]
    df_vector[m] = np.nan
    df_vector.loc[df_BI['filter']==True, m] = 1
    try:
        df_vector[m] = df_vector[m].fillna(0)
        df_vector[m] = df_vector[m].astype(int)
    except:
        continue
    del df_BI['filter']
    temp.to_csv('WF_FULL_TABLE/ALCL/'+m+'.txt', header=True, sep='|', index=False)
    print(m, ':', temp.shape[0])
    del temp

ALCL word frequecy:

ALCL : 907
ANAPLASTIC LARGE CELL LYMPHOMA : 493
LYMPHOMA : 1072
T-CELL LYMPHOMA : 74
B-CELL LYMPHOMA : 11
CANCER : 867
TUMOR : 167
SUSPECT : 447
CONFIRMED : 470
SUSPECT ALCL : 0
CONFIRMED ALCL : 22


In [37]:
print('Side word frequecy:\n')
for s in side:
    df_BI['filter'] = np.vectorize(text_search)(df_BI['FOI_TEXT'], s)
    temp = df_BI.loc[df_BI['filter']==True, :]
    df_vector[s] = np.nan
    df_vector.loc[df_BI['filter']==True, s] = 1
    try:
        df_vector[s] = df_vector[s].fillna(0)
        df_vector[s] = df_vector[s].astype(int)
    except:
        continue
    del df_BI['filter']
    temp.to_csv('WF_FULL_TABLE/side/'+s+'.txt', header=True, sep='|', index=False)
    print(s, ':', temp.shape[0])
    del temp

Side word frequecy:

LEFT : 6952
RIGHT : 7041
BILATERAL : 7314
BOTH SIDES : 0
LEFT+RIGHT : 3190


In [38]:
print('Biomarker word frequecy:\n')
for b in biomarker:
    df_BI['filter'] = np.vectorize(text_search)(df_BI['FOI_TEXT'], b)
    temp = df_BI.loc[df_BI['filter']==True, :]
    df_vector[b] = np.nan
    df_vector.loc[df_BI['filter']==True, b] = 1
    try:
        df_vector[b] = df_vector[b].fillna(0)
        df_vector[b] = df_vector[b].astype(int)
    except:
        continue
    del df_BI['filter']
    temp.to_csv('WF_FULL_TABLE/biomarker/'+b+'.txt', header=True, sep='|', index=False)
    print(b, ':', temp.shape[0])
    del temp

Biomarker word frequecy:

CD 30 : 7
CD30 : 177
ALK : 432
NEGATIVE : 359
POSITIVE : 569
CD30- : 8
CD30+ : 86
ALK- : 113
ALK+ : 0
CD30 NEGATIVE : 6
CD30 POSITIVE : 22
ALK NEGATIVE : 69
ALK POSITIVE : 0


In [46]:
print('Symptom word frequecy:\n')
for s in symptom:
    s = s.upper()
    df_BI['filter'] = np.vectorize(text_search)(df_BI['FOI_TEXT'], s)
    temp = df_BI.loc[df_BI['filter']==True, :]
    df_vector[s] = np.nan
    df_vector.loc[df_BI['filter']==True, s] = 1
    try:
        df_vector[s] = df_vector[s].fillna(0)
        df_vector[s] = df_vector[s].astype(int)
    except:
        continue
    del df_BI['filter']
    temp.to_csv('WF_FULL_TABLE/symptom/'+s+'.txt', header=True, sep='|', index=False)
    print(s, ':', temp.shape[0])
    del temp

Symptom word frequecy:

BREAST PAIN : 758
BREAST SWELL : 40
BREAST CYST : 8
BREAST CALCIF : 6
CAPSULAR CONTRACTURE : 3106
LYMPH NODE ENLARGE : 3
FIRMNESS OF BREAST : 0
HEMATOMA : 795
MASS : 627
LUMP : 713
RUPTURE : 6918
DEFLAT : 5102
INFECT : 2106
ABSCESS : 47
LEUKOPENIA : 2
NODULE : 132
SKIN DISCOLOR : 13
SKIN LESION : 17
SEROMA : 1141
EFFUSION : 45
FLUID : 684
EDEMA : 85
LEAK : 2053
RED : 7749
TENDER : 530
ERYTHEMA : 94
ASYMMETR : 716
BREAST ENLARGE : 17


In [47]:
df_vector.to_csv('keyword_vector.txt', sep='|', header=True, index=False)
df_vector.head(20)

,MDR_REPORT_KEY,MANUFACTURER_D_NAME,BRAND_NAME,GENERIC_NAME,FOI_TEXT,PMT CORP.,ALLERGAN,COX-UPHOFF INTERNATIONAL,MEDICAL ENGINEERING CORPRATION,POLY IMPLANT PROTHESE,...,FLUID,EDEMA,LEAK,RED,TENDER,ERYTHEMA,ASYMMETR,BREAST ENLARGE,DEFLATED,INFECTION
0,6730886,ALLERGAN,UNK SALINE IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",HEALTHCARE PROFESSIONAL REPORTED LEFT SIDE DEF...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6730886,ALLERGAN,UNK SALINE IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",DEVICE EVALUATION VISUAL ANALYSIS RETURNED DEV...,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,1
2,6734192,ALLERGAN,STYLE 68 SALINE FILLED BREAST IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",DEVICE REMAINS IMPLANTED REVIEW DEVICE HISTORY...,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,6734192,ALLERGAN,STYLE 68 SALINE FILLED BREAST IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",PATIENT REPORTED LEFT SIDE COLLAPSED ADDITIONA...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6283766,IDEAL IMPLANT INCORPORATED,IDEAL IMPLANT STRUCTURED BREAST IMPLANT,SALINE-FILLED BREAST IMPLANT,DEFLATION RESULTING EXPLANTATION,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6283766,IDEAL IMPLANT INCORPORATED,IDEAL IMPLANT STRUCTURED BREAST IMPLANT,SALINE-FILLED BREAST IMPLANT,,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6533466,MENTOR,MEMORY GEL BREAST IMPLANT,BREAST IMPLANT,BREAST AUGMENTATION B6 2015 SINCE DAY IVE SUFF...,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,6533466,MENTOR,MEMORY GEL BREAST IMPLANT,BREAST IMPLANT,BREAST AUGMENTATION B6 2015 SINCE DAY IVE SUFF...,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8,6315557,ALLERGAN,UNK MAMMARY IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",THERAPEUTIC GOODS ADMINISTRATION REPORTED CONF...,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,6315557,ALLERGAN,UNK MAMMARY IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",EVENT LYMPHOMA PHYSIOLOGICAL COMPLICATION ANAL...,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [48]:
print('Is there saline and silicone at the same time?')
fill_conflict = df_vector.loc[(df_vector['SALINE']==1) & (df_vector['SILICONE']==1), :]
fill_conflict.to_excel('fill_conflict.xlsx', 'fill_conflict', index=False)
fill_conflict.head()

Is there saline and silicone at the same time?


,MDR_REPORT_KEY,MANUFACTURER_D_NAME,BRAND_NAME,GENERIC_NAME,FOI_TEXT,PMT CORP.,ALLERGAN,COX-UPHOFF INTERNATIONAL,MEDICAL ENGINEERING CORPRATION,POLY IMPLANT PROTHESE,...,FLUID,EDEMA,LEAK,RED,TENDER,ERYTHEMA,ASYMMETR,BREAST ENLARGE,DEFLATED,INFECTION
30,6553778,DOW CORNING CORPRATION,BREAST IMPLANT,BREAST IMPLANT,SILICONE IMPLANT 1986 LAST YEAR NOTICED LEFT B...,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
31,6553778,DOW CORNING CORPRATION,BREAST IMPLANT,BREAST IMPLANT,SILICONE IMPLANT 1986 LAST YEAR NOTICED LEFT B...,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
32,6553778,DOW CORNING CORPRATION,BREAST IMPLANT,BREAST IMPLANT,SILICONE IMPLANT 1986 LAST YEAR NOTICED LEFT B...,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
39,6751940,ALLERGAN,UNK MAMMARY IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",EVENT KNOWN POTENTIAL ADVERSE EVENT ADDRESSED ...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53,6732140,ALLERGAN,UNK MAMMARY IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",INFORMATION REPORTER REGARDING RETURN DEVICE R...,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [49]:
print('Is there smooth and textured at the same time?')
surface_conflict = df_vector.loc[(df_vector['SMOOTH']==1) & (df_vector['TEXTURED']==1), :]
surface_conflict.to_excel('surface_conflict.xlsx', 'surface_conflict', index=False)
surface_conflict.head()

Is there smooth and textured at the same time?


,MDR_REPORT_KEY,MANUFACTURER_D_NAME,BRAND_NAME,GENERIC_NAME,FOI_TEXT,PMT CORP.,ALLERGAN,COX-UPHOFF INTERNATIONAL,MEDICAL ENGINEERING CORPRATION,POLY IMPLANT PROTHESE,...,FLUID,EDEMA,LEAK,RED,TENDER,ERYTHEMA,ASYMMETR,BREAST ENLARGE,DEFLATED,INFECTION
249,6498449,UNK,UNK MAMMARY IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",FDA REPORTS Â¿THE FDA RECEIVED TOTAL 359 MEDIC...,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
463,6567310,ALLERGAN,STYLE 410 COHESIVE SILICONE GEL FILLED BREAST ...,"PROSTHESIS, BREAST, NONINFLATABLE, INTERNAL, S...",REPORTED EVENT PATIENT LEFT SIDE Â¿NATRELLE ST...,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
514,6572184,ALLERGAN,BREAST IMPLANT SILICONE STYLE 120 600CC,BREAST IMPLANT,RIGHT OPEN CAPSULECTOMY IMPLANT REMOVAL DIAGNO...,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
515,6572184,ALLERGAN,"MENTOR SMOOTH ROUND PLUS STYLE, 1000 700CC",BREAST IMPLANT,RIGHT OPEN CAPSULECTOMY IMPLANT REMOVAL DIAGNO...,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
576,6585642,MENTOR,SMOOTH SALINE BREAST IMPLANTS,SMOOTH SALINE BREAST IMPLANTS,MENTOR COMBO GEL SILICONESALINE MAMMARY PROSTH...,0,0,0,0,0,...,0,0,1,1,0,0,0,0,1,1


In [50]:
print('Is there left and right at the same time?')
side_conflict = df_vector.loc[df_vector['LEFT+RIGHT']==1, :]
side_conflict.to_excel('side_conflict.xlsx', 'side_conflict', index=False)
side_conflict.head()

Is there left and right at the same time?


,MDR_REPORT_KEY,MANUFACTURER_D_NAME,BRAND_NAME,GENERIC_NAME,FOI_TEXT,PMT CORP.,ALLERGAN,COX-UPHOFF INTERNATIONAL,MEDICAL ENGINEERING CORPRATION,POLY IMPLANT PROTHESE,...,FLUID,EDEMA,LEAK,RED,TENDER,ERYTHEMA,ASYMMETR,BREAST ENLARGE,DEFLATED,INFECTION
0,6730886,ALLERGAN,UNK SALINE IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",HEALTHCARE PROFESSIONAL REPORTED LEFT SIDE DEF...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6734192,ALLERGAN,STYLE 68 SALINE FILLED BREAST IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",PATIENT REPORTED LEFT SIDE COLLAPSED ADDITIONA...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,6751925,ALLERGAN,STYLE 68 SALINE FILLED BREAST IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",NON-HEALTHCARE PROFESSIONAL REPORTED PATIENT E...,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
62,6725627,ALLERGAN,STYLE 68 SALINE FILLED BREAST IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",HEALTHCARE PROFESSIONAL REPORTED RIGHT SIDE PO...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63,6740437,ALLERGAN,STYLE 68 SALINE FILLED BREAST IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",HEALTHCARE PROFESSIONAL REPORTED LEFT SIDE DEF...,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
